In [1]:
import sys
import os

# 현재 작업 디렉토리를 프로젝트 루트 디렉토리로 설정

project_root = os.getcwd()
sys.path.append(project_root + '/modules')

from DB.chromadb_storing import ChromaDB
from langchain.schema import Document
from modules.utils import get_only_paragraphs, get_all_datas, create_documents
BASE_DIR = './modules/datas' # 데이터가 저장돼 있는 루트를 의미합니다.

df = get_only_paragraphs(BASE_DIR)
# df = get_all_datas(BASE_DIR) # 이미지와 테이블이 포함된 전체 데이터도 가져올 수 있습니다.
df[:3]


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,type,image_route,dir_route,file_name,page,investment,company_name,summary,table,original_content
5,12,paragraph,NaN,datas/SK하이닉스_교보증권(2024.10.25),SK하이닉스_교보증권(2024.10.25).pdf,1,교보증권(2024.10.25),SK하이닉스,SK하이닉스는 3Q24에 17.57조 원의 매출액과 7.03조 원의 영업이익을 기록...,NaN,"SK하이닉스의 3Q24 17.57조원(YoY +94%, QoQ +7%), 영업이익은..."
6,15,paragraph,NaN,datas/SK하이닉스_교보증권(2024.10.25),SK하이닉스_교보증권(2024.10.25).pdf,1,교보증권(2024.10.25),SK하이닉스,"SK하이닉스 4분기 매출액 19.9조원 (YoY+76%, QoQ+13%), 영업이익...",NaN,"4Q24 매출액 19.9조원(YoY+76%, QoQ+13%), 영업이익 8.3조원(..."
7,17,paragraph,NaN,datas/SK하이닉스_교보증권(2024.10.25),SK하이닉스_교보증권(2024.10.25).pdf,1,교보증권(2024.10.25),SK하이닉스,SK하이닉스는 HBM과 eSSD 등 프리미엄 제품의 높은 시장 지배력을 바탕으로 2...,NaN,시장 불확실성속에서도 HBM과 eSSD등 프리미엄 제품의 견고한 시장 지배력이 돋 ...


In [ ]:
# 데이터프레임의 행들을 각각 Document타입으로 바꾼 뒤, 리스트에 담습니다.
documents = create_documents(df)
documents[:3]

[Document(metadata={'id': 5, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/SK케미칼_DB금융투자(2023.11.20)', 'file_name': 'SK케미칼_DB금융투자(2023.11.20).pdf', 'page': 1, 'investment': 'DB금융투자(2023.11.20)', 'company_name': 'SK케미칼', 'table': nan, 'original_content': 'Eastman과 글로벌 Copolyester 시장 양분: Copolyester는 PET에 CHDM 원료를 사용해 만든 투 명 플라스틱으로 내화학성/내충격성/가공성이 뛰어나기에 PC/PMMA/PVC를 대체 가능하며 환경호 르몬(BPA), 발암물질(SM)을 사용하지 않기에 소비재 브랜드들의 선호도가 높다. Copolyester 시장은 Eastman과 SK케미칼의 과점체제이며 과점 시장의 특성상 두 회사 모두 정확한 생산능력 및 판매 량을 공유하지 않는다. Copolyester가 각광받는 분야는 화장품 및 패키징 분야로 화장품 부문 매출 비중이 30%를 상회하는 것으로 추정된다. 주목할 것은 수익성인데 전반적인 화학 불황에도 불구 23년 SK케미칼의 Copoly/유화 영업이익률은 12%, Eastman 전사 영업이익률은 12%로 견고하다.'}, page_content='SK케미칼은 투명 플라스틱인 Copolyester 시장에서 Eastman과 함께 과점 체제를 구축하고 있으며, 특히 화장품 및 패키징 분야에서 높은 인기를 얻고 있다. 이러한 제품들은 환경호르몬이나 발암물질을 사용하지 않아 소비재 브랜드들에게 선호되고 있으며, 23년에는 전체적인 화학 불황에도 불구하고 견조한 영업 이익률을 유지했다.'),
 Document(metadata={'id': 7, 'type': 'paragraph', 'image_route': nan, 'dir_ro

In [ ]:
# add_docs를 통해 문서들을 저장합니다.

collection_name = 'chrdb.db'
persist_directory = 'DB'
DB = ChromaDB(collection_name, persist_directory, mode = 'OpenAI_Emb')
#                   [가능한 모델]
                #   1. NaverCloudEmb
                #   2. KF_DebertaEmb
                #   3. Kure_Emb - 공개된 모델이 아님 (Access키 필요)
                #   4. OpenAI_Emb
DB.create_and_add(documents,)



In [ ]:
# verify_db를 통해 db내의 데이터 수를 확인할 수 있고, db객체를 받을 수 있습니다.
db = DB.verify_db()

In [3]:
# 데이터 로드
collection_name = 'chrdb.db'
persist_directory = 'DB'
DB = ChromaDB(collection_name, persist_directory,mode = 'NaverCloudEmb')

DB.load_collection()

Collection 'chrdb.db' loaded from 'DB/DB_NaverCloudEmb'.


/data/ephemeral/home/practicing/src/DB/chromadb_storing.py:105: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


In [4]:
# verify_db를 통해 db내의 데이터 수를 확인할 수 있고, db객체를 받을 수 있습니다.
db = DB.verify_db()

Total documents in collection 'chrdb.db': 388


In [7]:
import sqlite3
def inspect_sqlite_db(db_path):
    """
    Inspect the structure and contents of an SQLite3 database.
    
    :param db_path: Path to the SQLite3 database file.
    """
    try:
        # 데이터베이스 연결
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 모든 테이블 이름 가져오기
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(f"- {table[0]}")

        # 각 테이블의 스키마와 데이터를 확인
        for table in tables:
            table_name = table[0]
            print(f"\nSchema for table '{table_name}':")
            cursor.execute(f"PRAGMA table_info({table_name});")
            schema = cursor.fetchall()
            for column in schema:
                print(column)

            print(f"\nData in table '{table_name}':")
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
            rows = cursor.fetchall()
            for row in rows:
                print(row)

        # 연결 종료
        conn.close()
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")

In [ ]:
db_path = 'datas/'  # SQLite3 파일 경로
inspect_sqlite_db(db_path)

In [5]:
# 위처럼 import하여 kiwi tokenizer을 사용하는 bm25와 dpr, 앙상블 리트리버를 생성할 수 있습니다.
from retrievals import bm25, dpr, ensemble, llm_retriever

topk = 3

DPRRetriever = dpr(db, topk = 3)
BM25Retriever = bm25(documents, topk = 3)

retrievals = [DPRRetriever, BM25Retriever]
weights = [0.5, 0.5]
search_type = 'mmr'
ensemble_retriever = ensemble(retrievals, topk = topk, weights = weights, search_type = search_type)

llm_retriever = llm_retriever(documents, db, topk1= 10)



In [ ]:
# 다음과 같이 retriever.invoke(query)를 통해 관련 문서를 검색할 수 있습니다.
query = '네이버의 2024년 매출을 알려줘.'

DPRRetriever.invoke(query)

In [ ]:
BM25Retriever.invoke(query)

In [ ]:
# 어째서인지 Ensemble retrieval은 topk가 먹질 않습니다.. ㅋㅋ
ensemble_retriever.invoke(query)

In [ ]:
llm_retriever.invoke('네이버의 2024년 매출을 알려줘')

In [14]:
idx = list(map(int, result.content.split(', ')))
selected_docs = [docs[i] for i in idx]

In [ ]:
from evaluation import retrieval_evaluate
import pandas as pd
eval_dataset = pd.read_csv('datas/validation_dataset.csv')

# retrieval과 evaldataset을 통해 acc를 평가해볼 수 있습니다.
retrieval_evaluate(retrieval=DPRRetriever, eval_dataset = eval_dataset)

In [ ]:
selected_docs